## Data processing
This is the second iteration of [5-data-preparation](5-data-preparation.ipynb). We will be trying to use the `dask` library to extract the features. In order to do this we will be leveraging dask's parallelization ability by using the already defined functions for extracting fetures with `pandas`. We will have a two step tprocess:

1. Use dask's `apply` and `map_partitions` together with pandas feature extraction to extract features which are *trivially paralellizable*
2. Use dask's other methods to implement one-hot-encoding, frequency counting or normalization or any other kind of precedure that depends on the entire dataset

This method could be employed for future work.

In [1]:
%matplotlib inline
# standard library
import itertools
import sys, os
import re

from urllib.parse import urlparse

# pandas
import pandas as pd
import dask.dataframe as dd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# skelearn
from sklearn.preprocessing import LabelBinarizer

# local imports
sys.path.append(os.path.join(os.getcwd(), "src"))
from features import extract_features_from_df, extract_features_from_html

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [2]:
# read some data
df = pd.read_csv('../data/ecommerce-new/raw.csv', nrows=100)
df.head()

html  \
0  <!DOCTYPE html><html lang="ro" class=""><head>...   
1  <!DOCTYPE html><html xml:lang="ro" lang="ro" c...   
2  <!DOCTYPE html><html xmlns:og="http://ogp.me/n...   
3  <!DOCTYPE html><html lang="ro" class=""><head>...   
4  <!DOCTYPE html><html xml:lang="ro" lang="ro" c...   

                                                 url  
0  https://www.emag.ro/resigilate/placi_video/c?r...  
1  https://www.emag.ro/resigilate/ventilatoare-pc...  
2  https://www.olx.ro/auto-masini-moto-ambarcatiu...  
3                     https://www.emag.ro/resigilate  
4  https://www.emag.ro/label/pret,intre-200-si-50...

In [3]:
# extract the features
feats = extract_features_from_df(df, 5, 5)
feats.head()

depth  sibling_pos    tag  no_classes  has_id  no_children  has_text  \
0      1            0   html           0   False            2     False   
1      2            0   head           0   False           38     False   
2      3            0   meta           0   False            0     False   
3      3            1  title           0   False            0      True   
4      3            2   meta           0   False            0     False   

  classes  descendant1_no_nodes  descendant1_no_children_avg  \
0      []                     2                         83.0   
1      []                    38                          0.0   
2      []                     0                          0.0   
3      []                     0                          0.0   
4      []                     0                          0.0   

                         ...                          ancestor5_depth  \
0                        ...                                        0   
1                        ...                                        0   
2                        ...                                        0   
3                        ...                                        0   
4                        ...                                        0   

   ancestor5_sibling_pos  ancestor5_tag ancestor5_no_classes ancestor5_has_id  \
0                      0                                   0            False   
1                      0                                   0            False   
2                      0                                   0            False   
3                      0                                   0            False   
4                      0                                   0            False   

   ancestor5_no_children  ancestor5_has_text  ancestor5_classes  \
0                      0               False                 []   
1                      0               False                 []   
2                      0               False                 []   
3                      0               False                 []   
4                      0               False                 []   

                 path                                                url  
0               /html  https://www.emag.ro/resigilate/placi_video/c?r...  
1          /html/head  https://www.emag.ro/resigilate/placi_video/c?r...  
2  /html/head/meta[1]  https://www.emag.ro/resigilate/placi_video/c?r...  
3    /html/head/title  https://www.emag.ro/resigilate/placi_video/c?r...  
4  /html/head/meta[2]  https://www.emag.ro/resigilate/placi_video/c?r...  

[5 rows x 85 columns]

In [4]:
# check the columns for out candidates
feats.columns

Index(['depth', 'sibling_pos', 'tag', 'no_classes', 'has_id', 'no_children',
       'has_text', 'classes', 'descendant1_no_nodes',
       'descendant1_no_children_avg', 'descendant1_has_id_avg',
       'descendant1_no_classes_avg', 'descendant1_has_text_avg',
       'descendant1_classes', 'descendant1_tags', 'descendant2_no_nodes',
       'descendant2_no_children_avg', 'descendant2_has_id_avg',
       'descendant2_no_classes_avg', 'descendant2_has_text_avg',
       'descendant2_classes', 'descendant2_tags', 'descendant3_no_nodes',
       'descendant3_no_children_avg', 'descendant3_has_id_avg',
       'descendant3_no_classes_avg', 'descendant3_has_text_avg',
       'descendant3_classes', 'descendant3_tags', 'descendant4_no_nodes',
       'descendant4_no_children_avg', 'descendant4_has_id_avg',
       'descendant4_no_classes_avg', 'descendant4_has_text_avg',
       'descendant4_classes', 'descendant4_tags', 'descendant5_no_nodes',
       'descendant5_no_children_avg', 'descendant5_has_id

In [5]:
# make a dask dataframe out of it 
feats_ddf = dd.from_pandas(feats, chunksize=1000)
feats_ddf

Dask DataFrame Structure:
                 depth sibling_pos     tag no_classes has_id no_children has_text classes descendant1_no_nodes descendant1_no_children_avg descendant1_has_id_avg descendant1_no_classes_avg descendant1_has_text_avg descendant1_classes descendant1_tags descendant2_no_nodes descendant2_no_children_avg descendant2_has_id_avg descendant2_no_classes_avg descendant2_has_text_avg descendant2_classes descendant2_tags descendant3_no_nodes descendant3_no_children_avg descendant3_has_id_avg descendant3_no_classes_avg descendant3_has_text_avg descendant3_classes descendant3_tags descendant4_no_nodes descendant4_no_children_avg descendant4_has_id_avg descendant4_no_classes_avg descendant4_has_text_avg descendant4_classes descendant4_tags descendant5_no_nodes descendant5_no_children_avg descendant5_has_id_avg descendant5_no_classes_avg descendant5_has_text_avg descendant5_classes descendant5_tags ancestor1_depth ancestor1_sibling_pos ancestor1_tag ancestor1_no_classes ancestor1_has_id ancestor1_no_children ancestor1_has_text ancestor1_classes ancestor2_depth ancestor2_sibling_pos ancestor2_tag ancestor2_no_classes ancestor2_has_id ancestor2_no_children ancestor2_has_text ancestor2_classes ancestor3_depth ancestor3_sibling_pos ancestor3_tag ancestor3_no_classes ancestor3_has_id ancestor3_no_children ancestor3_has_text ancestor3_classes ancestor4_depth ancestor4_sibling_pos ancestor4_tag ancestor4_no_classes ancestor4_has_id ancestor4_no_children ancestor4_has_text ancestor4_classes ancestor5_depth ancestor5_sibling_pos ancestor5_tag ancestor5_no_classes ancestor5_has_id ancestor5_no_children ancestor5_has_text ancestor5_classes    path     url
npartitions=163                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
0                int64       int64  object      int64   bool       int64     bool  object                int64                     float64                float64                    float64                  float64              object           object                int64                     float64                float64                    float64                  float64              object           object                int64                     float64                float64                    float64                  float64              object           object                int64                     float64                float64                  

### One-hot encoding
Explore the posibility of one-hot encoding features

In [6]:
# try to categorize the tag names to be able to one-hot-encode
tag_cats = feats_ddf.categorize(columns=['tag'])
one_hot = dd.get_dummies(data=tag_cats.loc[:, 'tag'], prefix='tag').compute()

In [7]:
# SUCCESS!
one_hot.head()

tag_html  tag_head  tag_meta  tag_title  tag_link  tag_html_comment  \
0         1         0         0          0         0                 0   
1         0         1         0          0         0                 0   
2         0         0         1          0         0                 0   
3         0         0         0          1         0                 0   
4         0         0         1          0         0                 0   

   tag_script  tag_body  tag_noscript  tag_div    ...     tag_em  \
0           0         0             0        0    ...          0   
1           0         0             0        0    ...          0   
2           0         0             0        0    ...          0   
3           0         0             0        0    ...          0   
4           0         0             0        0    ...          0   

   tag_textarea  tag_address  tag_h5  tag_mktz-div  tag_omni-container  \
0             0            0       0             0                   0   
1             0            0       0             0                   0   
2             0            0       0             0                   0   
3             0            0       0             0                   0   
4             0            0       0             0                   0   

   tag_omni-ribb-cent  tag_omni-interact  tag_h6  tag_font  
0                   0                  0       0         0  
1                   0                  0       0         0  
2                   0                  0       0         0  
3                   0                  0       0         0  
4                   0                  0       0         0  

[5 rows x 66 columns]

### Frequency vectors
Experiment with obtaining frequency vectors from the lists of decendant tags

In [8]:
# compute all the tags available 
avail_tags = feats_ddf['descendant2_tags'].to_bag().map(set).fold(lambda x, y: x | y).compute()

In [9]:
def count_values(lst, values):
    """Given an iterable of values and one of keys, return the count of 
    the keys in the list(with 0 as default)"""
    count_dict = {val: 0 for val in values} # for overwriting with values
    for elem in lst:
        count_dict[elem] += 1
    return count_dict
    
count_values([1, 1, 2, 1], [1, 2, 3])

{1: 3, 2: 1, 3: 0}

In [10]:
# get the counts, and rename them with a prefix
freqcol_names = {tag_name: int for tag_name in avail_tags}

# returninga series from func creates more columns(this is a LIFE SAVER!)
freq_dicts = feats_ddf['descendant2_tags'].apply(lambda x: pd.Series(count_values(x, avail_tags)), meta=freqcol_names)
freq_dicts.loc[0, :].compute()

a  address  area  aside  b  br  button  caption  dd  div ...  style  sup  \
0  0        0     0      0  0   0       0        0   0    4 ...      0    0   

   table  tbody  td  textarea  th  title  tr  ul  
0      0      0   0         0   0      1   0   0  

[1 rows x 63 columns]

In [11]:
count_values(feats.loc[0, 'descendant2_tags'], avail_tags)

{'a': 0,
 'address': 0,
 'area': 0,
 'aside': 0,
 'b': 0,
 'br': 0,
 'button': 0,
 'caption': 0,
 'dd': 0,
 'div': 4,
 'dl': 0,
 'dt': 0,
 'em': 0,
 'fieldset': 0,
 'figcaption': 0,
 'figure': 0,
 'font': 0,
 'footer': 0,
 'form': 0,
 'h1': 0,
 'h2': 0,
 'h3': 0,
 'h4': 0,
 'h5': 0,
 'h6': 0,
 'header': 0,
 'html_comment': 4,
 'i': 0,
 'iframe': 0,
 'img': 1,
 'input': 107,
 'label': 0,
 'li': 0,
 'link': 12,
 'map': 0,
 'meta': 15,
 'mktz-div': 0,
 'nav': 0,
 'noindex': 0,
 'noscript': 1,
 'ol': 0,
 'omni-container': 0,
 'omni-interact': 0,
 'omni-ribb-cent': 0,
 'option': 0,
 'p': 0,
 's': 0,
 'script': 21,
 'section': 0,
 'select': 0,
 'small': 0,
 'span': 0,
 'strong': 0,
 'style': 0,
 'sup': 0,
 'table': 0,
 'tbody': 0,
 'td': 0,
 'textarea': 0,
 'th': 0,
 'title': 1,
 'tr': 0,
 'ul': 0}

### Implementation 
Now that we have our proof-of-concept implementations, we can do some more generic implementations to use with our data

In [12]:
def freq_vect_series(ser):
    """Given a series whose elements are python lists, return
    a dataframe where each record is the frequency vector for a certain
    element in the list. The columns will be prefixed with the series name
    
    Returns a dask datagrame."""
    # reduce all to a single set of tags
    avail_tags = ser.to_bag().fold(lambda a, b: a | set(b), set.union, initial=set()).compute()  
    # compute the frequencies of the given tags, pass the index as an argument to concat it to the dict
    # to preserv it 
    freqcol_names = {tag_name: int for tag_name in avail_tags}
    freqs = ser.apply(lambda x: pd.Series(count_values(x, avail_tags)), meta=freqcol_names)
    
    # rename the columns to be prefixed with the name of the series
    col_renames = {col_name: ser.name + '_' + col_name for col_name in avail_tags}
    return freqs.rename(columns=col_renames)
    

In [14]:
freq_vect = freq_vect_series(feats_ddf['descendant2_tags'])
freq_vect.head()

descendant2_tags_a  descendant2_tags_address  descendant2_tags_area  \
0                   0                         0                      0   
1                   0                         0                      0   
2                   0                         0                      0   
3                   0                         0                      0   
4                   0                         0                      0   

   descendant2_tags_aside  descendant2_tags_b  descendant2_tags_br  \
0                       0                   0                    0   
1                       0                   0                    0   
2                       0                   0                    0   
3                       0                   0                    0   
4                       0                   0                    0   

   descendant2_tags_button  descendant2_tags_caption  descendant2_tags_dd  \
0                        0                         0                    0   
1                        0                         0                    0   
2                        0                         0                    0   
3                        0                         0                    0   
4                        0                         0                    0   

   descendant2_tags_div         ...           descendant2_tags_style  \
0                     4         ...                                0   
1                     0         ...                                0   
2                     0         ...                                0   
3                     0         ...                                0   
4                     0         ...                                0   

   descendant2_tags_sup  descendant2_tags_table  descendant2_tags_tbody  \
0                     0                       0                       0   
1                     0                       0                       0   
2                     0                       0                       0   
3                     0                       0                       0   
4                     0                       0                       0   

   descendant2_tags_td  descendant2_tags_textarea  descendant2_tags_th  \
0                    0                          0                    0   
1                    0                          0                    0   
2                    0                          0                    0   
3                    0                          0                    0   
4                    0                          0                    0   

   descendant2_tags_title  descendant2_tags_tr  descendant2_tags_ul  
0                       1                    0                    0  
1                       0                    0                    0  
2                       0                    0                    0  
3                       0                    0                    0  
4                       0                    0                    0  

[5 rows x 63 columns]

In [15]:
# full dataframe implementation
def freq_vect_dataframe(ddf):
    """Given a dataframe of columns with python lists compute
    the merged dataframe of the frequency vectors returned
    by freq_vect_series."""
    ddfs = [freq_vect_series(ddf.loc[:, col_name]) for col_name in ddf.columns.tolist()]
    # basically compute all the frequency dataframes and returned the one-by-one merge result
    result = ddfs[0]
    for ddf in ddfs[1:]:
        result.assign(**{col_name: col_name for col_name in ddf.columns.tolist()})
    return result

In [16]:
freq_ddf = freq_vect_dataframe(feats_ddf.loc[:, ['descendant1_tags', 'descendant2_tags']])
freq_ddf.head()

descendant1_tags_a  descendant1_tags_address  descendant1_tags_area  \
0                   0                         0                      0   
1                   0                         0                      0   
2                   0                         0                      0   
3                   0                         0                      0   
4                   0                         0                      0   

   descendant1_tags_aside  descendant1_tags_b  descendant1_tags_body  \
0                       0                   0                      1   
1                       0                   0                      0   
2                       0                   0                      0   
3                       0                   0                      0   
4                       0                   0                      0   

   descendant1_tags_br  descendant1_tags_button  descendant1_tags_caption  \
0                    0                        0                         0   
1                    0                        0                         0   
2                    0                        0                         0   
3                    0                        0                         0   
4                    0                        0                         0   

   descendant1_tags_dd         ...           descendant1_tags_style  \
0                    0         ...                                0   
1                    0         ...                                0   
2                    0         ...                                0   
3                    0         ...                                0   
4                    0         ...                                0   

   descendant1_tags_sup  descendant1_tags_table  descendant1_tags_tbody  \
0                     0                       0                       0   
1                     0                       0                       0   
2                     0                       0                       0   
3                     0                       0                       0   
4                     0                       0                       0   

   descendant1_tags_td  descendant1_tags_textarea  descendant1_tags_th  \
0                    0                          0                    0   
1                    0                          0                    0   
2                    0                          0                    0   
3                    0                          0                    0   
4                    0                          0                    0   

   descendant1_tags_title  descendant1_tags_tr  descendant1_tags_ul  
0                       0                    0                    0  
1                       1                    0                    0  
2                       0                    0                    0  
3                       0                    0                    0  
4                       0                    0                    0  

[5 rows x 65 columns]

In [17]:
def one_hot_dataframe(ddf):
    """Given a dask dataframe encode its columns using one-hot. Every new column will
    be prefixed with the original name.
    
    Returns a dask dataframe."""
    tag_cats = ddf.categorize()  # converted to categoricals
    one_hot = dd.get_dummies(data=tag_cats, prefix=tag_cats.columns.tolist())
    return one_hot

one_hot_dataframe(feats_ddf.loc[:, ['tag', 'ancestor1_tag']]).head()

tag_html  tag_head  tag_meta  tag_title  tag_link  tag_html_comment  \
0         1         0         0          0         0                 0   
1         0         1         0          0         0                 0   
2         0         0         1          0         0                 0   
3         0         0         0          1         0                 0   
4         0         0         1          0         0                 0   

   tag_script  tag_body  tag_noscript  tag_div         ...          \
0           0         0             0        0         ...           
1           0         0             0        0         ...           
2           0         0             0        0         ...           
3           0         0             0        0         ...           
4           0         0             0        0         ...           

   ancestor1_tag_b  ancestor1_tag_figure  ancestor1_tag_figcaption  \
0                0                     0                         0   
1                0                     0                         0   
2                0                     0                         0   
3                0                     0                         0   
4                0                     0                         0   

   ancestor1_tag_em  ancestor1_tag_address  ancestor1_tag_mktz-div  \
0                 0                      0                       0   
1                 0                      0                       0   
2                 0                      0                       0   
3                 0                      0                       0   
4                 0                      0                       0   

   ancestor1_tag_omni-container  ancestor1_tag_omni-ribb-cent  \
0                             0                             0   
1                             0                             0   
2                             0                             0   
3                             0                             0   
4                             0                             0   

   ancestor1_tag_omni-interact  ancestor1_tag_font  
0                            0                   0  
1                            0                   0  
2                            0                   0  
3                            0                   0  
4                            0                   0  

[5 rows x 113 columns]

In [19]:
# WRAPPING IT ALL UP
def extract_features_from_ddf(ddf, depth, height):
    """Given a dask dataframe of the raw data, return the dask dataset containing all the
    extracted features and dropping the redundant ones."""
    feat_ddf = ddf.map_partitions(lambda df: extract_features_from_df(df, depth, height),
                                   meta=extract_features_from_df(pd.DataFrame(), depth, height)).clear_divisions()
    feat_ddf = feat_ddf.categorize(['url', 'path'])
    columns = feat_ddf.columns.tolist()  # used for filtering
    
    # one hot encoding
    one_hot_cols = list(filter(lambda col: re.match(r'.*tag$', col), columns))
    one_hot_ddf = one_hot_dataframe(feat_ddf.loc[:, one_hot_cols])

    # frequency vects
    freq_cols = list(filter(lambda col: re.match(r'descend.*tags$', col), columns))
    freq_ddf = freq_vect_dataframe(feat_ddf.loc[:, freq_cols])
    
    # drop redundant cols
    classes_cols =  list(filter(lambda col: re.match(r'^((descendant|ancestor)[0-9]+_)?classes$', col), columns))
    feat_ddf = feat_ddf.drop(one_hot_cols + freq_cols + classes_cols, axis='columns')
    return one_hot_ddf, freq_ddf, feat_ddf
    
oh, freqs, feats = extract_features_from_ddf(dd.from_pandas(df.iloc[:20], chunksize=10), 5, 5)

In [20]:
oh.compute()

tag_html  tag_head  tag_meta  tag_title  tag_link  tag_html_comment  \
0             1         0         0          0         0                 0   
1             0         1         0          0         0                 0   
2             0         0         1          0         0                 0   
3             0         0         0          1         0                 0   
4             0         0         1          0         0                 0   
5             0         0         1          0         0                 0   
6             0         0         1          0         0                 0   
7             0         0         0          0         1                 0   
8             0         0         1          0         0                 0   
9             0         0         1          0         0                 0   
10            0         0         1          0         0                 0   
11            0         0         1          0         0                 0   
12            0         0         1          0         0                 0   
13            0         0         1          0         0                 0   
14            0         0         1          0         0                 0   
15            0         0         1          0         0                 0   
16            0         0         1          0         0                 0   
17            0         0         1          0         0                 0   
18            0         0         0          0         1                 0   
19            0         0         0          0         1                 0   
20            0         0         1          0         0                 0   
21            0         0         0          0         1                 0   
22            0         0         0          0         1                 0   
23            0         0         0          0         1                 0   
24            0         0         0          0         1                 0   
25            0         0         0          0         1                 0   
26            0         0         0          0         1                 0   
27            0         0         0          0         1                 0   
28            0         0         0          0         1                 0   
29            0         0         0          0         0                 1   
...         ...       ...       ...        ...       ...               ...   
10852         0         0         0          0         0                 0   
10853         0         0         0          0         0                 0   
10854         0         0         0          0         0                 1   
10855         0         0         0          0         0                 0   
10856         0         0         0          0         0                 0   
10857         0         0         0          0         0                 0   
10858         0         0         0          0         0                 0   
10859         0         0         0          0         0                 0   
10860         0         0         0          0         0                 0   
10861         0         0         0          0         0                 0   
10862         0         0         0          0         0                 0   
10863         0         0         0          0         0                 0   
10864         0         0         0          0         0                 0   
10865         0         0         0          0         0                 0   
10866         0         0         0          0         0                 0   
10867         0         0         0          0         0                 0   
10868         0         0         0          0         0                 0   
10869         0         0         0          0         0                 0   
10870         0         0         0          0         0                 0   
10871         0         0    

In [21]:
freqs.compute()

descendant1_tags_a  descendant1_tags_address  descendant1_tags_area  \
0                       0                         0                      0   
1                       0                         0                      0   
2                       0                         0                      0   
3                       0                         0                      0   
4                       0                         0                      0   
5                       0                         0                      0   
6                       0                         0                      0   
7                       0                         0                      0   
8                       0                         0                      0   
9                       0                         0                      0   
10                      0                         0                      0   
11                      0                         0                      0   
12                      0                         0                      0   
13                      0                         0                      0   
14                      0                         0                      0   
15                      0                         0                      0   
16                      0                         0                      0   
17                      0                         0                      0   
18                      0                         0                      0   
19                      0                         0                      0   
20                      0                         0                      0   
21                      0                         0                      0   
22                      0                         0                      0   
23                      0                         0                      0   
24                      0                         0                      0   
25                      0                         0                      0   
26                      0                         0                      0   
27                      0                         0                      0   
28                      0                         0                      0   
29                      0                         0                      0   
...                   ...                       ...                    ...   
10852                   0                         0                      0   
10853                   0                         0                      0   
10854                   0                         0                      0   
10855                   0                         0                      0   
10856                   0                         0                      0   
10857                   0                         0                      0   
10858                   0                         0                      0   
10859                   0                         0                      0   
10860                   3                         0                      0   
10861                   0                         0                      0   
10862                   0                         0                      0   
10863                   0                         0                      0   
10864                   0                         0                      0   
10865                   0                         0                      0   
10866                   0                         0                      0   
10867                   0                         0                      0   
10868                   0                         0                      0   
10869                   0                         0                      0   
10870                   0                         0                      0   
10871                   0    

In [22]:
feats.compute()

depth  sibling_pos  no_classes  has_id  no_children  has_text  \
0          1            0           0   False            2     False   
1          2            0           0   False           38     False   
2          3            0           0   False            0     False   
3          3            1           0   False            0      True   
4          3            2           0   False            0     False   
5          3            3           0   False            0     False   
6          3            4           0   False            0     False   
7          3            5           0   False            0     False   
8          3            6           0   False            0     False   
9          3            7           0   False            0     False   
10         3            8           0   False            0     False   
11         3            9           0   False            0     False   
12         3           10           0   False            0     False   
13         3           11           0   False            0     False   
14         3           12           0   False            0     False   
15         3           13           0   False            0     False   
16         3           14           0   False            0     False   
17         3           15           0   False            0     False   
18         3           16           0   False            0     False   
19         3           17           0   False            0     False   
20         3           18           0   False            0     False   
21         3           19           0   False            0     False   
22         3           20           0   False            0     False   
23         3           21           0   False            0     False   
24         3           22           0   False            0     False   
25         3           23           0   False            0     False   
26         3           24           0   False            0     False   
27         3           25           0   False            0     False   
28         3           26           0   False            0     False   
29         3           27           0   False            0      True   
...      ...          ...         ...     ...          ...       ...   
10852      3           46           0    True            0      True   
10853      3           47           0    True            0      True   
10854      3           48           0   False            0      True   
10855      3           49           0    True            0     False   
10856      3           50           0    True            1     False   
10857      4            0           0   False            0     False   
10858      3           51           0    True            0     False   
10859      3           52           1    True            1     False   
10860      4            0           0    True            5     False   
10861      5            0           0    True            0     False   
10862      5            1           0    True            0     False   
10863      5            2           0    True            0     False   
10864      5            3           0    True            1     False   
10865      6            0           1    True            0     False   
10866      5            4           0    True            1     False   
10867      6            0           1    True            0     False   
10868      3           53           7    True            1     False   
10869      4            0           4   False            0     False   
10870      3           54           0   False            2     False   
10871      4            0           0   False            0      True   
10872      4            1           0   False            0     False   
10873      3           55           0   False            3     False   
10874      4            0           0   False            0      True   
10875      4            1           0   False  

In [38]:
feats.merge(oh, on=['url' ,'path']).merge(freqs, on=['url', 'path']).compute()

depth  sibling_pos  no_classes  has_id  no_children  has_text  \
0          1            0           1   False            2     False   
1          2            0           0   False           43     False   
2          3            0           0   False            0     False   
3          3            3           0   False            0      True   
4          3            5           0   False            0     False   
5          3            6           0   False            0     False   
6          3            7           0   False            0     False   
7          3            9           0   False            0     False   
8          3           11           0   False            0      True   
9          3           15           0   False            0     False   
10         3           20           0   False            0     False   
11         3           21           0   False            0     False   
12         3           22           0   False            0     False   
13         3           26           0   False            0      True   
14         3           28           0   False            0      True   
15         3           30           0   False            0     False   
16         3           33           0   False            0      True   
17         3           38           0   False            0      True   
18         3           39           0   False            0      True   
19         3           41           0   False            0      True   
20         3           42           0   False            0     False   
21         3            1           0   False            0      True   
22         3            3           0   False            0      True   
23         3            4           0   False            0      True   
24         3            5           0   False            0      True   
25         5            0           1    True            1     False   
26         9            0           1   False            0     False   
27         9            0           0   False            0     False   
28        11            0           1   False            1      True   
29        12            0           1   False            0     False   
...      ...          ...         ...     ...          ...       ...   
14229      3           16           0   False            0      True   
14230      3           18           0   False            0      True   
14231      3           19           0   False            0      True   
14232      3           20           0   False            0      True   
14233      3           22           0   False            0     False   
14234      3           23           0   False            0     False   
14235      3           24           0   False            0      True   
14236      3           26           0   False            0     False   
14237      3           28           0   False            0      True   
14238      3           30           0   False            0      True   
14239      3           31           0   False            0      True   
14240      3           33           0   False            0     False   
14241      3           34           0   False            0     False   
14242      1            0           0   False            1     False   
14243      2            0           0   False           37     False   
14244      3            0           0   False            0     False   
14245      3            5           0   False            0     False   
14246      3            8           0   False            0     False   
14247      3            9           0   False            0     False   
14248      3           10           0   False            0     False   
14249      3           12           0   False            0     False   
14250      3           15           0   False            0     False   
14251      3           17           0   False            0     False   
14252      3           21           0   False  